In [1]:
import math
import pandas as pd 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.init import xavier_normal_, xavier_uniform_
import argparse
import numpy as np
import time
import sys
from os.path import abspath
import random
import collections  
from collections import defaultdict
import scipy.sparse as sp
from itertools import product
from random import shuffle,randint,choice,sample
import csv 
from tqdm import tqdm

from util.conf import OptionConf
import torch
import torch.nn as nn 
import torch.nn.functional as F
from scipy.sparse import coo_matrix
from scipy.sparse.linalg import eigs
from util.loss_torch import bpr_loss, l2_reg_loss, EmbLoss, contrastLoss
from util.init import *
from base.torch_interface import TorchGraphInterface
import os
import numpy as np 
from torch.optim.lr_scheduler import ReduceLROnPlateau
import networkx as nx 
from torch_scatter import scatter_mean

from util.conf import ModelConf
from base.recommender import Recommender
from util.algorithm import find_k_largest
import time
from time import strftime, localtime
from data.loader import FileIO
from util.evaluation import ranking_evaluation,early_stopping
from data.data import Data
from data.graph import Graph
import multiprocessing
import heapq
import numpy as np
from sklearn.metrics import roc_auc_score
from prettytable import PrettyTable


/home/aiotlab3/anaconda3/envs/hungvv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
cores = multiprocessing.cpu_count() // 2
batch_test_flag = False
test_flag= 'full'

## Graph Recommender

In [24]:
class GraphRecommender(Recommender):
    def __init__(self, conf, data, data_kg, knowledge_set, **kwargs):
        super(GraphRecommender, self).__init__(conf, data, data_kg, knowledge_set,**kwargs)
        self.data = data
        self.data_kg = data_kg
        self.bestPerformance = []
        top = self.ranking['-topN'].split(',')
        self.Ks = [int(num) for num in top]
        self.max_N = max(self.Ks)
        
        self.output_path = kwargs['output_path']
        if not os.path.exists(self.output_path):
            os.makedirs(self.output_path)
            
    def print_model_info(self):
        super(GraphRecommender, self).print_model_info()
        # # print dataset statistics
        print('Training Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.training_size()))
        print('Test Set Size: (user number: %d, item number %d, interaction number: %d)' % (self.data.test_size()))
        print('=' * 80)

    def build(self):
        pass

    def train(self):
        pass

    def predict(self, u):
        pass


    def test(self, user_emb, item_emb):
        def process_bar(num, total):
            rate = float(num) / total
            ratenum = int(50 * rate)
            r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
            sys.stdout.write(r)
            sys.stdout.flush()

        # predict
        rec_list = {}
        user_count = len(self.data.test_set)
        lst_users =  list(self.data_kg.u2id.keys())
        lst_items =  list(self.data_kg.i2id.keys())
        
        for i, user in enumerate(self.data.test_set):
            user_id  = lst_users.index(user)
            score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
            candidates = score.cpu().numpy()
            
            # e_find_candidates = time.time()
            # print("Calculate candidates time: %f s" % (e_find_candidates - s_find_candidates))
            # predictedItems = denormalize(predictedItems, self.data.rScale[-1], self.data.rScale[0])
            rated_list, li = self.data.user_rated(user)
            for item in rated_list:
                candidates[lst_items.index(item)] = -10e8
            # s_find_k_largest = time.time()
            ids, scores = find_k_largest(self.max_N, candidates)
            # e_find_k_largest = time.time()
            # print("Find k largest candidates: %f s" % (e_find_k_largest - s_find_k_largest))
            item_names = [lst_items[iid] for iid in ids]
            rec_list[user] = list(zip(item_names, scores))
            if i % 1000 == 0:
                process_bar(i, user_count)
        process_bar(user_count, user_count)
        print('')
        return rec_list
    
    def evaluate(self, rec_list):
        self.recOutput.append('userId: recommendations in (itemId, ranking score) pairs, * means the item is hit.\n')
        for user in self.data.test_set:
            line = str(user) + ':'
            for item in rec_list[user]:
                line += ' (' + str(item[0]) + ',' + str(item[1]) + ')'
                if item[0] in self.data.test_set[user]:
                    line += '*'
            line += '\n'
            self.recOutput.append(line)
        current_time = strftime("%Y-%m-%d %H-%M-%S", localtime(time.time()))
        # output prediction result
        out_dir = self.output_path
        file_name = self.config['model.name'] + '@' + current_time + '-top-' + str(self.max_N) + 'items' + '.txt'
        FileIO.write_file(out_dir, file_name, self.recOutput)
        print('The result has been output to ', abspath(out_dir), '.')
        file_name = self.config['model.name'] + '@' + current_time + '-performance' + '.txt'
        self.result = ranking_evaluation(self.data.test_set, rec_list, self.topN)
        self.model_log.add('###Evaluation Results###')
        self.model_log.add(self.result)
        FileIO.write_file(out_dir, file_name, self.result)
        print('The result of %s:\n%s' % (self.model_name, ''.join(self.result)))

    def fast_evaluation(self, model, epoch, user_embed, item_embed, kwargs=None):
        print('Evaluating the model...')
        s_test = time.time()
        rec_list = self.test(user_embed, item_embed)
        e_test = time.time() 
        print("Test time: %f s" % (e_test - s_test))
        
        s_measure = time.time()
        measure = ranking_evaluation(self.data.test_set, rec_list, [self.max_N])
        e_measure = time.time()
        print("Measure time: %f s" % (e_measure - s_measure))
        
        if len(self.bestPerformance) > 0:
            count = 0
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            for k in self.bestPerformance[1]:
                if self.bestPerformance[1][k] > performance[k]:
                    count += 1
                else:
                    count -= 1
            if count < 0:
                self.bestPerformance[1] = performance
                self.bestPerformance[0] = epoch + 1
                # try:
                #     self.save(kwargs)
                # except:
                self.save(model)
        else:
            self.bestPerformance.append(epoch + 1)
            performance = {}
            for m in measure[1:]:
                k, v = m.strip().split(':')
                performance[k] = float(v)
            self.bestPerformance.append(performance)
            # try:
            #     self.save(kwargs)
            # except:
            self.save(model)
        print('-' * 120)
        print('Real-Time Ranking Performance ' + ' (Top-' + str(self.max_N) + ' Item Recommendation)')
        measure = [m.strip() for m in measure[1:]]
        print('*Current Performance*')
        print('Epoch:', str(epoch + 1) + ',', '  |  '.join(measure))
        bp = ''
        # for k in self.bestPerformance[1]:
        #     bp+=k+':'+str(self.bestPerformance[1][k])+' | '
        bp += 'Hit Ratio' + ':' + str(self.bestPerformance[1]['Hit Ratio']) + '  |  '
        bp += 'Precision' + ':' + str(self.bestPerformance[1]['Precision']) + '  |  '
        bp += 'Recall' + ':' + str(self.bestPerformance[1]['Recall']) + '  |  '
        # bp += 'F1' + ':' + str(self.bestPerformance[1]['F1']) + ' | '
        bp += 'NDCG' + ':' + str(self.bestPerformance[1]['NDCG'])
        print('*Best Performance* ')
        print('Epoch:fast_evaluation', str(self.bestPerformance[0]) + ',', bp)
        print('-' * 120)
        return measure
    
    def save(self, model):
        with torch.no_grad():
            user_emb, entity_emb = model.calc_cf_embeddings()
            item_emb = entity_emb[list(model.data_kg.id2i.keys())]
            self.best_user_emb, self.best_item_emb = user_emb, item_emb
        self.save_model(model)
    
    def save_model(self, model):
        # save model 
        current_time = strftime("%Y-%m-%d", localtime(time.time()))
        out_dir = self.output_path
        file_name =  self.config['model.name'] + '@' + current_time + '-weight' + '.pth'
        weight_file = out_dir + '/' + file_name 
        torch.save(model.state_dict(), weight_file)

    def save_performance_row(self, ep, data_ep):
        # opening the csv file in 'w' mode
        csv_path = self.output_path + 'train_performance.csv'
        
        # 'Hit Ratio:0.00328', 'Precision:0.00202', 'Recall:0.00337', 'NDCG:0.00292
        hit = float(data_ep[0].split(':')[1])
        precision = float(data_ep[1].split(':')[1])
        recall = float(data_ep[2].split(':')[1])
        ndcg = float(data_ep[3].split(':')[1])
        
        with open(csv_path, 'a+', newline = '') as f:
            header = ['ep', 'hit@20', 'prec@20', 'recall@20', 'ndcg@20']
            writer = csv.DictWriter(f, fieldnames = header)
            # writer.writeheader()
            writer.writerow({
                 'ep' : ep,
                 'hit@20': hit,
                 'prec@20': precision,
                 'recall@20': recall,
                 'ndcg@20': ndcg,
            })
            
    def save_loss_row(self, data_ep):
        csv_path = self.output_path + 'loss.csv'
        with open(csv_path, 'a+', newline ='') as f:
            header = ['ep', 'train_loss', 'cf_loss', 'kg_loss']
            writer = csv.DictWriter(f, fieldnames = header)
            # writer.writeheader()
            writer.writerow({
                'ep' : data_ep[0],
                'train_loss': data_ep[1],
                 'cf_loss': data_ep[2],
                 'kg_loss': data_ep[3]
            })

    def save_loss(self, train_losses, rec_losses, kg_losses):
        df_train_loss = pd.DataFrame(train_losses, columns = ['ep', 'loss'])
        df_rec_loss = pd.DataFrame(rec_losses, columns = ['ep', 'loss'])
        df_kg_loss = pd.DataFrame(kg_losses, columns = ['ep', 'loss'])
        df_train_loss.to_csv(self.output_path + '/train_loss.csv')
        df_rec_loss.to_csv(self.output_path + '/rec_loss.csv')
        df_kg_loss.to_csv(self.output_path + '/kg_loss.csv')
    
    def save_perfomance_training(self, log_train):
        df_train_log = pd.DataFrame(log_train)
        df_train_log.to_csv(self.output_path + '/train_performance.csv')

## Interaction

In [4]:
class Interaction(Data, Graph):
    def __init__(self, conf, training, test):
        self.conf = conf 
        Graph.__init__(self)
        Data.__init__(self,conf,training,test)

        self.user = {}
        self.item = {}
        self.id2user = {}
        self.id2item = {}

        self.training_set_u = defaultdict(dict)
        self.training_set_i = defaultdict(dict)
        self.test_set = defaultdict(dict)
        self.user_history_dict = defaultdict(dict)

        self.test_set_item = set()
        self.__generate_set()

        self.n_users = len(self.training_set_u)
        self.n_items = len(self.training_set_i) 

        self.n_cf_train = len(self.training_data)
        self.n_cf_test = len(self.test_data)
        
    def __generate_set(self):
        for entry in self.training_data:
            user, item, rating = entry
            user, item = int(user), int(item)
            if user not in self.user:
                self.user[user] = len(self.user)
                self.id2user[self.user[user]] = user
            if item not in self.item:
                self.item[item] = len(self.item)
                self.id2item[self.item[item]] = item
                # userList.append
            # construct user_history_dict 
            if rating == 1.0:
                if user not in self.user_history_dict:
                    self.user_history_dict[user] = []
                self.user_history_dict[user].append(item)

            self.training_set_u[user][item] = rating
            self.training_set_i[item][user] = rating
        
        for entry in self.test_data:
            user, item, rating = entry
            if user not in self.user:
                continue
            self.test_set[user][item] = rating
            self.test_set_item.add(item)

    def __create_sparse_bipartite_adjacency(self, self_connection=False):
        '''
        return a sparse adjacency matrix with the shape (user number + item number, user number + item number)
        '''
        n_nodes = self.n_users + self.n_items
        row_idx = [int(pair[0]) for pair in self.training_data]
        col_idx = [int(pair[1]) for pair in self.training_data]
        user_np = np.array(row_idx)
        item_np = np.array(col_idx)
        ratings = np.ones_like(user_np, dtype=np.float32)
        tmp_adj = sp.csr_matrix((ratings, (user_np, item_np + self.n_users)), shape=(n_nodes, n_nodes),dtype=np.float32)
        adj_mat = tmp_adj + tmp_adj.T
        if self_connection:
            adj_mat += sp.eye(n_nodes)
        return adj_mat
    
    def __create_sparse_interaction_matrix(self):
        """
            return a sparse adjacency matrix with the shape (user number, item number)
        """
        row, col, entries = [], [], []
        for pair in self.training_data:
            row += [int(pair[0])]
            col += [int(pair[1])]
            entries += [1.0]
        interaction_mat = sp.csr_matrix((entries, (row, col)), shape=(self.n_users,self.n_items),dtype=np.float32)
        inv_interaction_mat = sp.csr_matrix((entries, (col, row)), shape=(self.n_items, self.n_users), dtype=np.float32)
        return interaction_mat, inv_interaction_mat
            
    def get_user_id(self, u):
        if u in self.user:
            return self.user[u]

    def get_item_id(self, i):
        if i in self.item:
            return self.item[i]

    def training_size(self):
        return len(self.user), len(self.item), len(self.training_data)

    def test_size(self):
        return len(self.test_set), len(self.test_set_item), len(self.test_data)

    def contain(self, u, i):
        'whether user u rated item i'
        if u in self.user and i in self.training_set_u[u]:
            return True
        else:
            return False

    def contain_user(self, u):
        'whether user is in training set'
        if u in self.user:
            return True
        else:
            return False

    def contain_item(self, i):
        """whether item is in training set"""
        if i in self.item:
            return True
        else:
            return False

    def user_rated(self, u):
        return list(self.training_set_u[u].keys()), list(self.training_set_u[u].values())

    def item_rated(self, i):
        return list(self.training_set_i[i].keys()), list(self.training_set_i[i].values())


## Knowledge

In [5]:
class Knowledge:
    def __init__(self, conf, training, test, knowledge, inverse_r=True):
        self.conf = conf 

        self.inverse_r = inverse_r
        self.kg_data = knowledge

        self.relation = {}
        self.id2rel = {}
        
        self.cf_train_data = np.array(training)
        self.cf_test_data = np.array(test)
        
        self.training_set_e = defaultdict(dict)
        self.train_user_set = defaultdict(list)
        self.test_user_set = defaultdict(list)

        self.u2id = {}
        self.id2u = {}
        self.i2id = {}
        self.id2i = {}
        
        self.remap_item()
        self.user_dict = {
            'train_user_set': self.train_user_set,
            'test_user_set': self.test_user_set
        }
        self.triplets = self.read_triplets()

        # get data 
        self.graph, relation_dict = self.build_graph()
        self.adj_mat, self.norm_mat_list, self.mean_mat_list = self.__create_sparse_relational_graph(relation_dict)
        # self.kg_interaction_mat = self.__create_sparse_knowledge_interaction_matrix()

    def remap_item(self):
        min_user = min(min(self.cf_train_data[:,0]), min(self.cf_test_data[:,0]))
        self.n_users =  int(max(max(self.cf_train_data[:,0]), max(self.cf_test_data[:,0])) - min(min(self.cf_train_data[:,0]), min(self.cf_test_data[:,0])) + 2)
        
        for u_id, i_id, _ in self.cf_train_data:
            self.train_user_set[int(u_id)].append(int(i_id))
            if int(u_id) not in self.u2id:
                self.u2id[int(u_id)] = int(u_id- min_user) 
                self.id2u[int(u_id- min_user)] = int(u_id)
            if int(i_id) not in self.i2id:
                self.i2id[int(i_id)] = len(self.i2id)
                self.id2i[self.i2id[int(i_id)]] = int(i_id)
            
        for u_id, i_id, _ in self.cf_test_data:
            self.test_user_set[int(u_id)].append(int(i_id))
            if int(u_id) not in self.u2id:
                self.u2id[int(u_id)] = int(u_id- min_user) 
                self.id2u[int(u_id- min_user)] = int(u_id)
            if int(i_id) not in self.i2id:
                self.i2id[int(i_id)] = len(self.i2id)
                self.id2i[self.i2id[int(i_id)]] = int(i_id)
        self.n_items = len(self.i2id)
                
    def read_triplets(self):
        can_triplets_np = self.kg_data.to_numpy(dtype=np.int32)
        can_triplets_np = np.unique(can_triplets_np, axis=0)
    
        if self.inverse_r:
            # get triplets with inverse direction like <entity, is-aspect-of, item>
            inv_triplets_np = can_triplets_np.copy()
            inv_triplets_np[:, 0] = can_triplets_np[:, 2]
            inv_triplets_np[:, 2] = can_triplets_np[:, 0]
            inv_triplets_np[:, 1] = can_triplets_np[:, 1] + max(can_triplets_np[:, 1]) + 1
            # consider two additional relations --- 'interact' and 'be interacted'
            can_triplets_np[:, 1] = can_triplets_np[:, 1] + 1
            inv_triplets_np[:, 1] = inv_triplets_np[:, 1] + 1
            # get full version of knowledge graph
            triplets = np.concatenate((can_triplets_np, inv_triplets_np), axis=0)
        else:
            # consider two additional relations --- 'interact'.
            can_triplets_np[:, 1] = can_triplets_np[:, 1] + 1
            triplets = can_triplets_np.copy()
        self.n_entities = int(max(max(triplets[:, 0]), max(triplets[:, 2]))) + 1  # including items + users
        self.n_nodes = int(self.n_entities + self.n_users) 
        self.n_relations = int(max(triplets[:, 1]) + 1)
        return triplets
        
    def build_graph(self):
        print("Building the graph ...")
        ckg_graph = nx.MultiDiGraph()
        rd = collections.defaultdict(list)

        print("Begin to load interaction triplets")
        for u_id, i_id, _ in tqdm(self.cf_train_data, ascii=True):
            rd[0].append([u_id, i_id])
            
        print("Begin to load knowledge triplets")
        for h_id, r_id, t_id in tqdm(self.triplets, ascii=True):
            ckg_graph.add_edge(h_id, t_id, key=r_id)
            rd[r_id].append([h_id, t_id])
        return ckg_graph, rd

    def __create_sparse_relational_graph(self, relation_dict):
        def _bi_norm_lap(adj):
            # D^{-1/2}AD^{-1/2}
            rowsum = np.array(adj.sum(1))
    
            d_inv_sqrt = np.power(rowsum, -0.5).flatten()
            d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
            d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    
            # bi_lap = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt)
            bi_lap = d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt)
            return bi_lap.tocoo()
    
        def _si_norm_lap(adj):
            # D^{-1}A
            rowsum = np.array(adj.sum(1))
    
            d_inv = np.power(rowsum, -1).flatten()
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)
    
            norm_adj = d_mat_inv.dot(adj)
            return norm_adj.tocoo()
    
        adj_mat_list = []
        print("Begin to build sparse relation matrix ...")
        for r_id in tqdm(relation_dict.keys()):
            np_mat = np.array(relation_dict[r_id])
            if r_id == 0:
                cf = np_mat.copy()
                # cf[:, 1] = cf[:, 1] + self.n_users  # [0, n_items) -> [n_users, n_users+n_items)
                vals = [1.] * len(cf)
                adj = sp.coo_matrix((vals, (cf[:, 0], cf[:, 1])), shape=(self.n_nodes, self.n_nodes))
            else:
                vals = [1.] * len(np_mat)
                adj = sp.coo_matrix((vals, (np_mat[:, 0], np_mat[:, 1])), shape=(self.n_nodes, self.n_nodes))
            adj_mat_list.append(adj)
    
        norm_mat_list = [_bi_norm_lap(mat) for mat in adj_mat_list]
        mean_mat_list = [_si_norm_lap(mat) for mat in adj_mat_list]
        # interaction: user->item, [n_users, n_entities]
        norm_mat_list[0] = norm_mat_list[0].tocsr()[self.n_entities:, :self.n_entities].tocoo()
        mean_mat_list[0] = mean_mat_list[0].tocsr()[self.n_entities:, :self.n_entities].tocoo()
        return adj_mat_list, norm_mat_list, mean_mat_list 
  

## Layers

In [6]:
class Aggregator(nn.Module):
    """
    Relational Path-aware Convolution Network
    """
    def __init__(self, n_users, n_factors):
        super(Aggregator, self).__init__()
        self.n_users = n_users
        self.n_factors = n_factors

    def forward(self, entity_emb, user_emb, latent_emb,
                edge_index, edge_type, interact_mat,
                weight, disen_weight_att):

        n_entities = entity_emb.shape[0]
        channel = entity_emb.shape[1]
        n_users = self.n_users
        n_factors = self.n_factors

        """KG aggregate"""
        head, tail = edge_index
        head = head.to(device)
        tail = tail.to(device)
        entity_emb = entity_emb.to(device)
        
        edge_relation_emb = weight[edge_type - 1].to(device)  # exclude interact, remap [1, n_relations) to [0, n_relations-1)
        neigh_relation_emb = entity_emb[tail] * edge_relation_emb  # [-1, channel]
        entity_agg = scatter_mean(src=neigh_relation_emb, index=head, dim_size=n_entities, dim=0)
        """cul user->latent factor attention"""
        score_ = torch.mm(user_emb, latent_emb.t())
        score = nn.Softmax(dim=1)(score_).unsqueeze(-1).to(device)  # [n_users, n_factors, 1]

        """user aggregate"""
        user_agg = torch.sparse.mm(interact_mat, entity_emb).to(device)  # [n_users, channel]
        disen_weight = torch.mm(nn.Softmax(dim=-1)(disen_weight_att), weight).expand(int(n_users), n_factors, channel).to(device)
        user_agg = user_agg * (disen_weight * score).sum(dim=1) + user_agg  # [n_users, channel]

        return entity_agg, user_agg

In [7]:
class GraphConv(nn.Module):
    """
    Graph Convolutional Network
    """
    def __init__(self, channel, n_hops, n_users,
                 n_factors, n_relations, interact_mat,
                 ind, node_dropout_rate=0.5, mess_dropout_rate=0.1):
        super(GraphConv, self).__init__()

        self.convs = nn.ModuleList().to(device)
        self.interact_mat = interact_mat
        self.n_relations = n_relations
        self.n_users = n_users
        self.n_factors = n_factors
        self.node_dropout_rate = node_dropout_rate
        self.mess_dropout_rate = mess_dropout_rate
        self.ind = ind

        self.temperature = 0.2

        initializer = nn.init.xavier_uniform_
        weight = initializer(torch.empty(n_relations - 1, channel))  # not include interact
        self.weight = nn.Parameter(weight).to(device)  # [n_relations - 1, in_channel]

        disen_weight_att = initializer(torch.empty(n_factors, n_relations - 1))
        self.disen_weight_att = nn.Parameter(disen_weight_att).to(device)

        for i in range(n_hops):
            self.convs.append(Aggregator(n_users=n_users, n_factors=n_factors).to(device))

        self.dropout = nn.Dropout(p=mess_dropout_rate)  # mess dropout

    def _edge_sampling(self, edge_index, edge_type, rate=0.5):
        # edge_index: [2, -1]
        # edge_type: [-1]
        n_edges = edge_index.shape[1]
        random_indices = np.random.choice(n_edges, size=int(n_edges * rate), replace=False)
        return edge_index[:, random_indices], edge_type[random_indices]

    def _sparse_dropout(self, x, rate=0.5):
        noise_shape = x._nnz()

        random_tensor = rate
        random_tensor += torch.rand(noise_shape).to(x.device)
        dropout_mask = torch.floor(random_tensor).type(torch.bool)
        i = x._indices()
        v = x._values()

        i = i[:, dropout_mask]
        v = v[dropout_mask]

        out = torch.sparse.FloatTensor(i, v, x.shape).to(x.device)
        return out * (1. / (1 - rate))
        
    def _cul_cor(self):
        def CosineSimilarity(tensor_1, tensor_2):
            # tensor_1, tensor_2: [channel]
            normalized_tensor_1 = tensor_1 / tensor_1.norm(dim=0, keepdim=True)
            normalized_tensor_2 = tensor_2 / tensor_2.norm(dim=0, keepdim=True)
            return (normalized_tensor_1 * normalized_tensor_2).sum(dim=0) ** 2  # no negative
        def DistanceCorrelation(tensor_1, tensor_2):
            # tensor_1, tensor_2: [channel]
            # ref: https://en.wikipedia.org/wiki/Distance_correlation
            channel = tensor_1.shape[0]
            zeros = torch.zeros(channel, channel).to(tensor_1.device)
            zero = torch.zeros(1).to(tensor_1.device)
            tensor_1, tensor_2 = tensor_1.unsqueeze(-1), tensor_2.unsqueeze(-1)
            """cul distance matrix"""
            a_, b_ = torch.matmul(tensor_1, tensor_1.t()) * 2, \
                   torch.matmul(tensor_2, tensor_2.t()) * 2  # [channel, channel]
            tensor_1_square, tensor_2_square = tensor_1 ** 2, tensor_2 ** 2
            a, b = torch.sqrt(torch.max(tensor_1_square - a_ + tensor_1_square.t(), zeros) + 1e-8), \
                   torch.sqrt(torch.max(tensor_2_square - b_ + tensor_2_square.t(), zeros) + 1e-8)  # [channel, channel]
            """cul distance correlation"""
            A = a - a.mean(dim=0, keepdim=True) - a.mean(dim=1, keepdim=True) + a.mean()
            B = b - b.mean(dim=0, keepdim=True) - b.mean(dim=1, keepdim=True) + b.mean()
            dcov_AB = torch.sqrt(torch.max((A * B).sum() / channel ** 2, zero) + 1e-8)
            dcov_AA = torch.sqrt(torch.max((A * A).sum() / channel ** 2, zero) + 1e-8)
            dcov_BB = torch.sqrt(torch.max((B * B).sum() / channel ** 2, zero) + 1e-8)
            return dcov_AB / torch.sqrt(dcov_AA * dcov_BB + 1e-8)
            
        def MutualInformation():
            # disen_T: [num_factor, dimension]
            disen_T = self.disen_weight_att.t()

            # normalized_disen_T: [num_factor, dimension]
            normalized_disen_T = disen_T / disen_T.norm(dim=1, keepdim=True)

            pos_scores = torch.sum(normalized_disen_T * normalized_disen_T, dim=1)
            ttl_scores = torch.sum(torch.mm(disen_T, self.disen_weight_att), dim=1)

            pos_scores = torch.exp(pos_scores / self.temperature)
            ttl_scores = torch.exp(ttl_scores / self.temperature)

            mi_score = - torch.sum(torch.log(pos_scores / ttl_scores))
            return mi_score

        """cul similarity for each latent factor weight pairs"""
        if self.ind == 'mi':
            return MutualInformation()
        else:
            cor = 0
            for i in range(self.n_factors):
                for j in range(i + 1, self.n_factors):
                    if self.ind == 'distance':
                        cor += DistanceCorrelation(self.disen_weight_att[i], self.disen_weight_att[j])
                    else:
                        cor += CosineSimilarity(self.disen_weight_att[i], self.disen_weight_att[j])
        return cor

    def forward(self, user_emb, entity_emb, latent_emb, edge_index, edge_type,
                interact_mat, mess_dropout=True, node_dropout=False):

        """node dropout"""
        if node_dropout:
            edge_index, edge_type = self._edge_sampling(edge_index, edge_type, self.node_dropout_rate)
            interact_mat = self._sparse_dropout(interact_mat, self.node_dropout_rate)

        entity_res_emb = entity_emb  # [n_entity, channel]
        user_res_emb = user_emb  # [n_users, channel]
        cor = self._cul_cor()
        for i in range(len(self.convs)):
            entity_emb, user_emb = self.convs[i](entity_emb, user_emb, latent_emb,
                                                 edge_index, edge_type, interact_mat,
                                                 self.weight, self.disen_weight_att)

            """message dropout"""
            if mess_dropout:
                entity_emb = self.dropout(entity_emb)
                user_emb = self.dropout(user_emb)
            entity_emb = F.normalize(entity_emb)
            user_emb = F.normalize(user_emb)

            """result emb"""
            entity_res_emb = torch.add(entity_res_emb, entity_emb)
            user_res_emb = torch.add(user_res_emb, user_emb)

        return entity_res_emb, user_res_emb, cor

## Model

In [8]:
class KGIN(nn.Module):
    def __init__(self, args, rec, device=None):
        super(KGIN, self).__init__()

        self.data_kg = rec.data_kg 
        self.n_users = self.data_kg.n_users
        self.n_relations = self.data_kg.n_relations
        self.n_entities = self.data_kg.n_entities  # include items
        self.n_nodes = rec.data_kg.n_nodes # n_users + n_entities

        self.decay = args['l2']
        self.sim_decay = args['sim_regularity']
        self.emb_size = args['dim']
        self.context_hops = args['context_hops']
        self.n_factors = args['n_factors']
        self.node_dropout = args['node_dropout']
        self.node_dropout_rate = args['node_dropout_rate']
        self.mess_dropout = args['mess_dropout']
        self.mess_dropout_rate = args['mess_dropout_rate']
        self.ind = args['ind']
        self.device = device

        self.adj_mat = self.data_kg.mean_mat_list[0]
        self.graph = self.data_kg.graph
        self.edge_index, self.edge_type = self._get_edges(self.graph)

        self._init_weight()
        self.all_embed = nn.Parameter(self.all_embed).to(self.device)
        self.latent_emb = nn.Parameter(self.latent_emb).to(self.device)

        self.gcn = self._init_model()

    def _init_weight(self):
        initializer = nn.init.xavier_uniform_
        self.all_embed = initializer(torch.empty(self.n_nodes, self.emb_size).to(self.device))
        self.latent_emb = initializer(torch.empty(self.n_factors, self.emb_size).to(self.device))

        # [n_users, n_entities]
        self.interact_mat = self._convert_sp_mat_to_sp_tensor(self.adj_mat).to(self.device)

    def _init_model(self):
        return GraphConv(channel=self.emb_size,
                         n_hops=self.context_hops,
                         n_users=self.n_users,
                         n_relations=self.n_relations,
                         n_factors=self.n_factors,
                         interact_mat=self.interact_mat,
                         ind=self.ind,
                         node_dropout_rate=self.node_dropout_rate,
                         mess_dropout_rate=self.mess_dropout_rate)

    def _convert_sp_mat_to_sp_tensor(self, X):
        coo = X.tocoo()
        i = torch.LongTensor([coo.row, coo.col])
        v = torch.from_numpy(coo.data).float()
        return torch.sparse.FloatTensor(i, v, coo.shape)

    def _get_indices(self, X):
        coo = X.tocoo()
        return torch.LongTensor([coo.row, coo.col]).t()  # [-1, 2]

    def _get_edges(self, graph):
        graph_tensor = torch.tensor(list(graph.edges))  # [-1, 3]
        index = graph_tensor[:, :-1]  # [-1, 2]
        type = graph_tensor[:, -1]  # [-1, 1]
        return index.t().long().to(self.device), type.long().to(self.device)

    def forward(self, batch=None):
        user =  torch.LongTensor([ rec.data_kg.u2id[u.item()] for u in  batch['users'] ] ).to(device)
        pos_item = batch['pos_items']
        neg_item = batch['neg_items']
        user_emb = self.all_embed[self.n_entities:, :]
        item_emb = self.all_embed[:self.n_entities, :]
        # entity_gcn_emb: [n_entity, channel]
        # user_gcn_emb: [n_users, channel]
        entity_gcn_emb, user_gcn_emb, cor =  self.gcn(user_emb,
                                                     item_emb,
                                                     self.latent_emb,
                                                     self.edge_index,
                                                     self.edge_type,
                                                     self.interact_mat,
                                                     mess_dropout=self.mess_dropout,
                                                     node_dropout=self.node_dropout)
        u_e = user_gcn_emb[user]
        pos_e, neg_e = entity_gcn_emb[pos_item], entity_gcn_emb[neg_item]
        return self.create_bpr_loss(u_e, pos_e, neg_e, cor)

    def generate(self):
        user_emb = self.all_embed[self.n_entities:, :]
        item_emb = self.all_embed[:self.n_entities, :]
        return self.gcn(user_emb,
                        item_emb,
                        self.latent_emb,
                        self.edge_index,
                        self.edge_type,
                        self.interact_mat,
                        mess_dropout=False, node_dropout=False)[:-1]

    def rating(self, u_g_embeddings, i_g_embeddings):
        return torch.matmul(u_g_embeddings, i_g_embeddings.t())

    def create_bpr_loss(self, users, pos_items, neg_items, cor):
        batch_size = users.shape[0]
        pos_scores = torch.sum(torch.mul(users, pos_items), axis=1)
        neg_scores = torch.sum(torch.mul(users, neg_items), axis=1)

        mf_loss = -1 * torch.mean(nn.LogSigmoid()(pos_scores - neg_scores))

        # cul regularizer
        regularizer = (torch.norm(users) ** 2
                       + torch.norm(pos_items) ** 2
                       + torch.norm(neg_items) ** 2) / 2
        emb_loss = self.decay * regularizer / batch_size
        cor_loss = self.sim_decay * cor

        return mf_loss + emb_loss + cor_loss, mf_loss, emb_loss, cor

    def generate_kg_drop(self):
        user_emb = self.all_embed[self.n_entities:, :]
        item_emb = self.all_embed[:self.n_entities, :]
        edge_index, edge_type = self.gcn._edge_sampling(self.edge_index, self.edge_type, self.kg_drop_test_keep_rate)
        return self.gcn(user_emb,
                        item_emb,
                        self.latent_emb,
                        edge_index,
                        edge_type,
                        self.interact_mat,
                        mess_dropout=False, node_dropout=False)[:-1]

    def calc_cf_embeddings(self):
        user_emb = self.all_embed[self.n_entities:, :]
        item_emb = self.all_embed[:self.n_entities, :]
        return user_emb, item_emb

## Sampler

In [9]:
def get_feed_dict(train_entity_pairs, start, end, train_user_set, n_items):

    def negative_sampling(user_item, train_user_set, n_items):
        neg_items = []
        for user, _, _ in user_item:
            user = int(user)
            while True:
                neg_item = np.random.randint(low=0, high=n_items, size=1)[0]
                if neg_item not in train_user_set[user]:
                    break
            neg_items.append(neg_item)
        return neg_items

    feed_dict = {}
    entity_pairs = np.array(train_entity_pairs[start:end])
    feed_dict['users'] = torch.LongTensor(entity_pairs[:, 0]).to(device)
    feed_dict['pos_items'] = torch.LongTensor(entity_pairs[:, 1]).to(device)
    feed_dict['neg_items'] = torch.LongTensor(negative_sampling(entity_pairs, train_user_set, n_items)).to(device)
    return feed_dict

## Util

In [10]:
def _L2_loss_mean(x):
    return torch.mean(torch.sum(torch.pow(x, 2), dim=1, keepdim=False) / 2.)

In [11]:
def ranklist_by_heapq(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = 0.
    return r, auc

def get_auc(item_score, user_pos_test):
    item_score = sorted(item_score.items(), key=lambda kv: kv[1])
    item_score.reverse()
    item_sort = [x[0] for x in item_score]
    posterior = [x[1] for x in item_score]

    r = []
    for i in item_sort:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = AUC(ground_truth=r, prediction=posterior)
    return auc

def ranklist_by_sorted(user_pos_test, test_items, rating, Ks):
    item_score = {}
    for i in test_items:
        item_score[i] = rating[i]

    K_max = max(Ks)
    K_max_item_score = heapq.nlargest(K_max, item_score, key=item_score.get)

    r = []
    for i in K_max_item_score:
        if i in user_pos_test:
            r.append(1)
        else:
            r.append(0)
    auc = get_auc(item_score, user_pos_test)
    return r, auc

def get_performance(user_pos_test, r, auc, Ks):
    precision, recall, ndcg, hit_ratio = [], [], [], []

    for K in Ks:
        precision.append(precision_at_k(r, K))
        recall.append(recall_at_k(r, K, len(user_pos_test)))
        ndcg.append(ndcg_at_k(r, K, user_pos_test))
        hit_ratio.append(hit_at_k(r, K))

    return {'recall': np.array(recall), 'precision': np.array(precision),
            'ndcg': np.array(ndcg), 'hit_ratio': np.array(hit_ratio), 'auc': auc}

In [12]:
def recall(rank, ground_truth, N):
    return len(set(rank[:N]) & set(ground_truth)) / float(len(set(ground_truth)))


def precision_at_k(r, k):
    """Score is precision @ k
    Relevance is binary (nonzero is relevant).
    Returns:
        Precision @ k
    Raises:
        ValueError: len(r) must be >= k
    """
    assert k >= 1
    r = np.asarray(r)[:k]
    return np.mean(r)


def average_precision(r,cut):
    """Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).
    Returns:
        Average precision
    """
    r = np.asarray(r)
    out = [precision_at_k(r, k + 1) for k in range(cut) if r[k]]
    if not out:
        return 0.
    return np.sum(out)/float(min(cut, np.sum(r)))


def mean_average_precision(rs):
    """Score is mean average precision
    Relevance is binary (nonzero is relevant).
    Returns:
        Mean average precision
    """
    return np.mean([average_precision(r) for r in rs])


def dcg_at_k(r, k, method=1):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, ground_truth, method=1):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Returns:
        Normalized discounted cumulative gain

        Low but correct defination
    """
    GT = set(ground_truth)
    if len(GT) > k :
        sent_list = [1.0] * k
    else:
        sent_list = [1.0]*len(GT) + [0.0]*(k-len(GT))
    dcg_max = dcg_at_k(sent_list, k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


def recall_at_k(r, k, all_pos_num):
    r = np.asfarray(r)[:k]
    return np.sum(r) / all_pos_num


def hit_at_k(r, k):
    r = np.array(r)[:k]
    if np.sum(r) > 0:
        return 1.
    else:
        return 0.

def F1(pre, rec):
    if pre + rec > 0:
        return (2.0 * pre * rec) / (pre + rec)
    else:
        return 0.

def AUC(ground_truth, prediction):
    try:
        res = roc_auc_score(y_true=ground_truth, y_score=prediction)
    except Exception:
        res = 0.
    return res

## Train

In [28]:
def train(model, rec, args):
    # seed
    random.seed(args['seed'])
    np.random.seed(args['seed'])
    torch.manual_seed(args['seed'])
    torch.cuda.manual_seed_all(args['seed'])

    lst_train_losses = []
    lst_rec_losses = []
    lst_cor_losses = []
    lst_performances = []
    recall_list = []
    
    len_cf =  rec.data.n_cf_train
    optimizer  = torch.optim.Adam(model.parameters(), lr=lRate)

    # kg_data = rec.data_kg.kg_train_data.to_numpy()
    train_cf_data = rec.data.training_data
    user_dict = rec.data_kg.user_dict
    
    for ep in range(maxEpoch):
        model.train()
        
        train_losses = []
        cf_losses = []
        cor_losses = []
        
        cf_total_loss = 0
        cor_total_loss = 0
        
        # shuffle(kg_data)
        shuffle(train_cf_data)

        loss, s, cor_loss = 0,0,0 
        train_s_t  = time.time()

        with tqdm(total= len_cf // batchSize) as pbar:
            while s + batchSize <= len_cf:
                batch = get_feed_dict(train_cf_data, s, s+ batchSize, user_dict['train_user_set'], rec.data.n_items)
                batch_loss, _, _, batch_cor = model(batch)

                batch_loss = batch_loss
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()

                loss += batch_loss
                cor_loss += batch_cor
                s += args['batch_size']
                pbar.update(1)

                cf_losses.append(batch_loss.item())
                cor_losses.append(cor_loss.item()) 

        cf_loss = np.mean(cf_losses)
        cor_loss = np.mean(cor_losses)
        train_loss = cf_loss + cor_loss

        train_e_t = time.time()
        
        """testing"""
        with torch.no_grad():
            user_emb, entity_emb = train_model.calc_cf_embeddings()
            item_emb = entity_emb[list(train_model.data_kg.id2i.keys())]
            data_ep = rec.fast_evaluation(model, ep, user_emb, item_emb)

            # early stopping when cur_best_pre_0 is decreasing for ten successive steps.
            cur_recall = float(data_ep[2].split(':')[1])
            recall_list.append(cur_recall)
            best_recall, should_stop = early_stopping(recall_list, 100)
            if should_stop:
                break
            # """save weight"""
            # if ret['recall'][0] == cur_best_pre_0 and args.save:
            #     torch.save(model.state_dict(), args['weight_path'])
            # else:
            # logging.info('training loss at epoch %d: %f' % (epoch, loss.item()))
        print('using time %.4f, training loss at epoch %d: %.4f, cor: %.6f' % (train_e_t - train_s_t, ep, loss.item(), cor_loss.item()))
        
        rec.save_performance_row(ep, data_ep)
        rec.save_loss_row([ep, train_loss, cf_loss, cor_loss])
    
        lst_performances.append(data_ep)
        lst_train_losses.append([ep, train_loss]) 
        lst_rec_losses.append([ep, cf_loss])
        lst_cor_losses.append([ep, cor_loss])
        
    rec.save_loss(lst_train_losses, lst_rec_losses, lst_cor_losses)
    rec.save_perfomance_training(lst_performances)
    print('early stopping at %d, recall@20:%.4f' % (ep, best_recall))
    user_emb, item_emb = rec.best_user_emb, rec.best_item_emb
    return user_emb, item_emb 

## Test

In [14]:
def test(rec, user_emb, item_emb):
    def process_bar(num, total):
        rate = float(num) / total
        ratenum = int(50 * rate)
        r = '\rProgress: [{}{}]{}%'.format('+' * ratenum, ' ' * (50 - ratenum), ratenum*2)
        sys.stdout.write(r)
        sys.stdout.flush()

    # predict
    rec_list = {}
    user_count = len(rec.data.test_set)
    for i, user in enumerate(rec.data.test_set):
        user_id = rec.data_kg.u2id[user]
        score = torch.matmul(user_emb[user_id], item_emb.transpose(0, 1))
        candidates = score.cpu().numpy()
        
        rated_list, li = rec.data.user_rated(user)
        for item in rated_list:
            candidates[rec.data_kg.i2id[item]] = -10e8
        # s_find_k_largest = time.time()
        ids, scores = find_k_largest(rec.max_N, candidates)

        item_names = [rec.data_kg.id2i[iid] for iid in ids]
        rec_list[user] = list(zip(item_names, scores))
        if i % 1000 == 0:
            process_bar(i, user_count)
    process_bar(user_count, user_count)
    print('')
    rec.evaluate(rec_list)

## Main

In [15]:
model = 'KGIN'
config = ModelConf('./conf/' + model + '.conf')
lRates = [0.01]
lRateKGs = [0.01]
lrDecays = [0.9]
maxEpochs = [1]
batchSizes = [2048]
batchSizeKGs = [8192]
nLayers = [2]
regs = [0.1]
regkgs = [ 1e-5]
embeddingSizes = [128]
datasets = ['lastfm']
sim_regularities = [1e-4]
context_hopss =  [2]
n_factors= [4]
node_dropouts= [True]
node_dropout_rates= [0.5]
inds = ['distance']
inverse_r = True

In [16]:
dataset ='lastfm'
training_data = FileIO.load_data_set('./dataset/' + dataset + '/' +config['training.set'], config['model.type'])
test_data = FileIO.load_data_set('./dataset/' + dataset + '/'  +config['test.set'], config['model.type'])
knowledge_set = FileIO.load_kg_data('./dataset/' + dataset +'/'+ dataset +'.kg')

In [17]:
data = Interaction(config, training_data, test_data)
data_kg = Knowledge(config, training_data, test_data, knowledge_set, inverse_r)

Building the graph ...
Begin to load interaction triplets


100%|###########################################################################################| 69624/69624 [00:00<00:00, 958871.19it/s]


Begin to load knowledge triplets


100%|#######################################################################################| 1301288/1301288 [00:05<00:00, 221273.91it/s]


Begin to build sparse relation matrix ...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 74.56it/s]
/tmp/ipykernel_1725154/3369743934.py:102: RuntimeWarning: divide by zero encountered in power
  d_inv_sqrt = np.power(rowsum, -0.5).flatten()
/tmp/ipykernel_1725154/3369743934.py:114: RuntimeWarning: divide by zero encountered in power
  d_inv = np.power(rowsum, -1).flatten()


In [29]:
hyperparameters = [lRates, lRateKGs, lrDecays, maxEpochs, batchSizes, batchSizeKGs, nLayers, regs, embeddingSizes, datasets, sim_regularities, context_hopss, n_factors, node_dropouts, node_dropout_rates, inds]
for params in product(*hyperparameters):
    lRate, lRateKG, lrDecay, maxEpoch, batchSize, batchSizeKG, nLayer, reg, embeddingSize, dataset, sim_regularity, context_hop, n_factor, node_dropout, node_dropout_rate, ind = params
    args = {
        'lr': lRate,
        'max_epoch': maxEpoch,
        'batch_size': batchSize, 
        'lr_decay': lrDecay,
        'dataset': dataset,
        'n_layers': nLayer,
        'dim': embeddingSize,
        'l2': reg,
        'sim_regularity': sim_regularity,
        'mess_dropout': '[0.1, 0.1, 0.1]',
        'mess_dropout_rate': 0.1,
        'context_hops': context_hop,
        'n_factors': n_factor,
        'node_dropout': node_dropout, 
        'node_dropout_rate': node_dropout_rate,
        'ind': ind,
        'inverse_r': inverse_r,
        'seed': 123,
    }

    args['output_path'] =  f"./results/{model}/{dataset}/@{model}-emb:{args['dim']}-bs:{args['batch_size']}-lr:{args['lr']}-n_layers:{args['n_layers']}-reg:{args['l2']}-sim_regularity:{args['sim_regularity']}/"
    if not os.path.exists(args['output_path']):
        os.makedirs(args['output_path'])

    current_time = strftime("%Y-%m-%d", localtime(time.time()))
    file_name =  config['model.name'] + '@' + current_time + '-weight' + '.pth'
    weight_path = args['output_path'] + file_name 
    args['weight_path'] = weight_path
    # data
    # training_data = FileIO.load_data_set('./dataset/' + dataset + '/' +config['training.set'], config['model.type'])
    # test_data = FileIO.load_data_set('./dataset/' + dataset + '/'  +config['test.set'], config['model.type'])
    # knowledge_set = FileIO.load_kg_data('./dataset/' + dataset +'/'+ dataset +'.kg')
    # data = Interaction(config, training_data, test_data)
    # data_kg = Knowledge(config, training_data, test_data, knowledge_set)
    # rec 
    rec = GraphRecommender(config, data, data_kg, knowledge_set, **args)
    # A_in = TorchGraphInterface.convert_sparse_mat_to_tensor(rec.data_kg.kg_interaction_mat).cuda()
    train_model = KGIN(args, rec, device=device)
    user_emb, item_emb = train(train_model, rec, args)   
    test(rec, user_emb, item_emb)

parameter ss_rate is not found in the configuration file!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:26<00:00,  1.24it/s]

Evaluating the model...
Progress: [                                                  ]0%

Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.679310 s
Measure time: 0.016283 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio:0.00414  |  Precision:0.00255  |  Recall:0.00418  |  NDCG:0.00316
*Best Performance* 
Epoch:fast_evaluation 1, Hit Ratio:0.00414  |  Precision:0.00255  |  Recall:0.00418  |  NDCG:0.00316
------------------------------------------------------------------------------------------------------------------------
using time 26.7208, training loss at epoch 0: 25.6492, cor: 30.217722


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:26<00:00,  1.23it/s]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:29<00:00,  1.11it/s]

Evaluating the model...
Progress: [                                                  ]0%

Progress: [++++++++++++++++++++++++++++++++++++++++++++++++++]100%
Test time: 1.714116 s
Measure time: 0.015692 s
------------------------------------------------------------------------------------------------------------------------
Real-Time Ranking Performance  (Top-20 Item Recommendation)
*Current Performance*
Epoch: 5, Hit Ratio:0.0034  |  Precision:0.0021  |  Recall:0.00327  |  NDCG:0.00281
*Best Performance* 
Epoch:fast_evaluation 2, Hit Ratio:0.00504  |  Precision:0.00311  |  Recall:0.0054  |  NDCG:0.00386
------------------------------------------------------------------------------------------------------------------------
using time 29.8024, training loss at epoch 4: 17.1772, cor: 30.217722


 33%|█████████████████████████████████▋                                                                   | 11/33 [00:10<00:21,  1.02it/s]


KeyboardInterrupt: 